In [0]:
!pip install geopandas
!pip install geopy

In [0]:
try:
  # Use the %tensorflow_version magic if in colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

In [0]:
import numpy as np
import os
from google.colab import files
import zipfile
from geopy.extra.rate_limiter import RateLimiter
import pandas as pd

In [0]:
!pip install kaggle

print("Please upload your Kaggle API token file (ie. kaggle.json)")
uploaded = files.upload()

! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d ted8080/house-prices-and-images-socal
with zipfile.ZipFile("house-prices-and-images-socal.zip","r") as zip_ref:
    zip_ref.extractall("./")

In [0]:
import pandas as pd

housing_csv = pd.read_csv("./socal2.csv")
housing_csv.head()

In [0]:
def concat_address(street, city):
  return street + " " + city

housing_csv['address'] = housing_csv.apply(lambda x: concat_address(street = x['street'], city = x['citi']), axis=1)

housing_csv = housing_csv.drop(columns=filtered_cols)

In [0]:
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
def process_addr(addr, recursion=0):
  try:
    p_addr = geocode(addr)
    print("Completed processing:", addr)
    return p_addr
  except GeocoderTimedOut as e:
      if recursion > 10:
          raise e
      time.sleep(1)
      return geocode(city, recursion=recursion + 1)

housing_csv['location'] = housing_csv['address'].apply(process_addr)
housing_csv['point'] = housing_csv['location'].apply(lambda loc: tuple(loc.point) if loc else None)
housing_csv[['latitude', 'longitude', 'altitude']] = housing_csv.DataFrame(df['point'].tolist(), index=housing_csv.index)

# Remove unnecessary columns from dataset
filtered_cols = [
  'citi',
  'street',
  'altitude',
  'location', 
  'point'
]